In [ ]:
## https://github.com/divamgupta/image-segmentation-keras/blob/master/Models/FCN8.py

In [ ]:
from keras.models import *
from keras.layers import *


# crop o1 wrt o2
def crop( o1 , o2 , i  ):
	o_shape2 = Model( i  , o2 ).output_shape
	outputHeight2 = o_shape2[1]
	outputWidth2 = o_shape2[2]

	o_shape1 = Model( i  , o1 ).output_shape
	outputHeight1 = o_shape1[1]
	outputWidth1 = o_shape1[2]

	cx = abs( outputWidth1 - outputWidth2 )
	cy = abs( outputHeight2 - outputHeight1 )

	if outputWidth1 > outputWidth2:
		o1 = Cropping2D( cropping=((0,0) ,  (  0 , cx )), data_format=IMAGE_ORDERING  )(o1)
	else:
		o2 = Cropping2D( cropping=((0,0) ,  (  0 , cx )), data_format=IMAGE_ORDERING  )(o2)
	
	if outputHeight1 > outputHeight2 :
		o1 = Cropping2D( cropping=((0,cy) ,  (  0 , 0 )), data_format=IMAGE_ORDERING  )(o1)
	else:
		o2 = Cropping2D( cropping=((0, cy ) ,  (  0 , 0 )), data_format=IMAGE_ORDERING  )(o2)

	return o1 , o2 

def FCN88( nClasses ,  input_height = 375 , input_width = 1242 , vgg_level=3):

	# assert input_height%32 == 0
	# assert input_width%32 == 0

	# https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_th_dim_ordering_th_kernels.h5
	img_input = Input(shape=(input_height,input_width,3))

	x = Conv2D(64, (3, 3),  padding='same', name='block1_conv1', data_format=IMAGE_ORDERING )(img_input)
	x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', data_format=IMAGE_ORDERING )(x)
	x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool', data_format=IMAGE_ORDERING )(x)
	f1 = x
	# Block 2
	x = Conv2D(128, (3, 3),  padding='same', name='block2_conv1', data_format=IMAGE_ORDERING )(x)
	x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', data_format=IMAGE_ORDERING )(x)
	x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool', data_format=IMAGE_ORDERING )(x)
	f2 = x

	# Block 3
	x = Conv2D(256, (3, 3),  padding='same', name='block3_conv1', data_format=IMAGE_ORDERING )(x)
	x = Conv2D(256, (3, 3),  padding='same', name='block3_conv2', data_format=IMAGE_ORDERING )(x)
	x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3', data_format=IMAGE_ORDERING )(x)
	x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool', data_format=IMAGE_ORDERING )(x)
	f3 = x

	# Block 4
	x = Conv2D(512, (3, 3),  padding='same', name='block4_conv1', data_format=IMAGE_ORDERING )(x)
	x = Conv2D(512, (3, 3),  padding='same', name='block4_conv2', data_format=IMAGE_ORDERING )(x)
	x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3', data_format=IMAGE_ORDERING )(x)
	x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool', data_format=IMAGE_ORDERING )(x)
	f4 = x

	# Block 5
	x = Conv2D(512, (3, 3),  padding='same', name='block5_conv1', data_format=IMAGE_ORDERING )(x)
	x = Conv2D(512, (3, 3),  padding='same', name='block5_conv2', data_format=IMAGE_ORDERING )(x)
	x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3', data_format=IMAGE_ORDERING )(x)
	x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool', data_format=IMAGE_ORDERING )(x)
	f5 = x

	#x = Flatten(name='flatten')(x)
	#x = Dense(4096, activation='relu', name='fc1')(x)
	#x = Dense(4096, activation='relu', name='fc2')(x)
	#x = Dense( 1000 , activation='softmax', name='predictions')(x)

	vgg  = Model(  img_input , x  )
	vgg.load_weights(VGG_Weights_path)

	o = f5

	#o = ( Conv2D( 4096 , ( 7 , 7 ) , activation='relu' , padding='same', data_format=IMAGE_ORDERING))(o)
	#o = Dropout(0.5)(o)
	#o = ( Conv2D( 4096 , ( 1 , 1 ) , activation='relu' , padding='same', data_format=IMAGE_ORDERING))(o)
	#o = Dropout(0.5)(o)

	o = ( Conv2D( nClasses ,  ( 1 , 1 ) ,kernel_initializer='he_normal' , data_format=IMAGE_ORDERING))(o)
	o = Conv2DTranspose( nClasses , kernel_size=(4,4) ,  strides=(2,2) , use_bias=False, data_format=IMAGE_ORDERING )(o)

	o2 = f4
	o2 = ( Conv2D( nClasses ,  ( 1 , 1 ) ,kernel_initializer='he_normal' , data_format=IMAGE_ORDERING))(o2)
	
	o , o2 = crop( o , o2 , img_input )
	
	o = Add()([ o , o2 ])

	o = Conv2DTranspose( nClasses , kernel_size=(4,4) ,  strides=(2,2) , use_bias=False, data_format=IMAGE_ORDERING )(o)
	o2 = f3 
	o2 = ( Conv2D( nClasses ,  ( 1 , 1 ) ,kernel_initializer='he_normal' , data_format=IMAGE_ORDERING))(o2)
	o2 , o = crop( o2 , o , img_input )
	o  = Add()([ o2 , o ])


	o = Conv2DTranspose( nClasses , kernel_size=(16,16) ,  strides=(8,8) , use_bias=False, data_format=IMAGE_ORDERING )(o)
	
	o_shape = Model(img_input , o ).output_shape
	
	outputHeight = o_shape[1]
	outputWidth = o_shape[2]

	o = (Activation('softmax'))(o)

	#extra_width =  outputWidth   

    
	o = Cropping2D( cropping=((1,0) ,  (3 , 3)), data_format=IMAGE_ORDERING  )(o)
    
	model = Model( img_input , o )
    
	model.outputWidth = outputWidth
	model.outputHeight = outputHeight

	return model


#if __name__ == '__main__':
#	m = FCN8( 101 )
#	from keras.utils import plot_model
#	plot_model( m , show_shapes=True , to_file='model.png')


In [ ]:
dir_data = "KITTI_MOD_fixed"
dir_seg = dir_data + "/training/mask/"
dir_img = dir_data + "/training/images/"
dir_seg_test = dir_data + "/testing/mask/"
dir_img_test = dir_data + "/testing/images/"

In [ ]:
import cv2, os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
## seaborn has white grid by default so I will get rid of this.
sns.set_style("whitegrid", {'axes.grid' : False})


ldseg = np.array(os.listdir(dir_seg))
## pick the first image file
fnm = ldseg[208]
print(fnm)

## read in the original image and segmentation labels
seg = cv2.imread(dir_seg + fnm )/255 # (360, 480, 3)
img_is = cv2.imread(dir_img + fnm )
print("seg.shape={}, img_is.shape={}".format(seg.shape,img_is.shape))

## Check the number of labels
mi, ma = np.min(seg), np.max(seg)
n_classes = 2
print("minimum seg = {}, maximum seg = {}, Total number of segmentation classes = {}".format(mi,ma, n_classes))

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1,1,1)
ax.imshow(img_is)
ax.set_title("original image")
plt.show()

fig = plt.figure(figsize=(15,10))
for k in range(n_classes):
    ax = fig.add_subplot(3,n_classes,k+1)
    ax.imshow((seg == k)*1.0)
    ax.set_title("label = {}".format(k))


plt.show()

In [ ]:
import random
def give_color_to_seg_img(seg,n_classes):
    '''
    seg : (input_width,input_height,3)
    '''
    
    if len(seg.shape)==3:
        seg = seg[:,:,0]
    seg_img = np.zeros( (seg.shape[0],seg.shape[1],3) ).astype('float')
    colors = sns.color_palette("hls", n_classes)
    
    for c in range(n_classes):
        segc = (seg == c)
        seg_img[:,:,0] += (segc*( colors[c][0] ))
        seg_img[:,:,1] += (segc*( colors[c][1] ))
        seg_img[:,:,2] += (segc*( colors[c][2] ))

    return(seg_img)

input_height , input_width = 375, 1242 #384, 1248  #288, 960 #384, 1248 # 192 , 640 ##
output_height , output_width = 375, 1242 #384, 1248 #288, 960   #384, 1248 # 192 , 640 ##


ldseg = np.array(os.listdir(dir_seg))
for fnm in ldseg[np.random.choice(len(ldseg),3,replace=False)]:
    fnm = fnm.split(".")[0]
    seg = cv2.imread(dir_seg + fnm + ".png") # (360, 480, 3)
    img_is = cv2.imread(dir_img + fnm + ".png")
    seg_img = give_color_to_seg_img(seg,n_classes)

    fig = plt.figure(figsize=(20,40))
    ax = fig.add_subplot(1,4,1)
    ax.imshow(seg_img)
    
    ax = fig.add_subplot(1,4,2)
    ax.imshow(img_is/255.0)
    ax.set_title("original image {}".format(img_is.shape[:2]))
    
    ax = fig.add_subplot(1,4,3)
    ax.imshow(seg_img)
    
    ax = fig.add_subplot(1,4,4)
    ax.imshow(img_is)
    ax.set_title("resized to {}".format((output_height , output_width)))
    plt.show()

In [ ]:
def getImageArr( path , width , height ):
        img = cv2.imread(path, 1)
        img = cv2.resize(img, (width,height))
        img = np.float32(img) / 127.5 - 1
        return img

def getSegmentationArr( path , nClasses ,  width , height  ):

    seg_labels = np.zeros((  height , width  , nClasses ))
    img = cv2.imread(path, 1)
    img = cv2.resize(img, (width,height))
    #img = cv2.copyMakeBorder( img, 1, 0, 3, 3, cv2.BORDER_CONSTANT )
    img = img[:, : , 0]/255

    for c in range(nClasses):
        seg_labels[: , : , c ] = (img == c).astype(int)
    ##seg_labels = np.reshape(seg_labels, ( width*height,nClasses  ))
    return seg_labels

In [ ]:
from keras.applications.vgg16 import preprocess_input

images = os.listdir(dir_img)
images.sort()
segmentations  = os.listdir(dir_seg)
segmentations.sort()
    
count = 0    
    
X = []
Y = []
for im , seg in zip(images,segmentations) :
    X.append( getImageArr(dir_img + im , input_width , input_height )  )
    Y.append( getSegmentationArr( dir_seg + seg , n_classes , output_width , output_height )  )

X, Y = np.array(X) , np.array(Y)
#X = preprocess_input(X)
print(X.shape,Y.shape)

In [ ]:
Y[1,:,:,0]

In [ ]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
import keras, sys, time, warnings
from keras.models import *
from keras.layers import *
import pandas as pd 
warnings.filterwarnings("ignore")

In [ ]:
VGG_Weights_path = "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

n_classes= 2
IMAGE_ORDERING =  "channels_last" 

In [ ]:
model = FCN88( nClasses = n_classes , input_height = input_height, input_width = input_width )


from keras.utils import plot_model
plot_model(model, to_file='model_no 404.png', show_shapes=True, show_layer_names=True)

In [ ]:
def focal_loss(alpha=0.25,gamma=2.0):
    def focal_crossentropy(y_true, y_pred):
        bce = K.binary_crossentropy(y_true, y_pred)
        
        y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
        p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))
        
        alpha_factor = 1
        modulating_factor = 1

        alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
        modulating_factor = K.pow((1-p_t), gamma)

        # compute the final loss and return
        return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
    return focal_crossentropy

In [ ]:
from keras import optimizers

sgd = optimizers.SGD(lr=1E-2 , decay=5**(-4), momentum=0.9, nesterov=True)
#adam = optimizers.Adam(lr=1e-2)

#sgd = optimizers.SGD(lr=1)

model.compile(loss = 'categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

hist = model.fit(X,Y, batch_size=4 ,epochs=20,  shuffle = True, verbose=1)

In [ ]:
for key in ['loss']:
    plt.plot(hist.history[key],label=key)
plt.legend()
plt.show()

In [ ]:
model.save('FCN_last.h5')
model.save_weights('FCN_last_weights.h5')

In [ ]:
from keras.models import load_model
model = load_model('FCN_last.h5')

In [ ]:
model.evaluate()

In [ ]:
images = os.listdir(dir_img_test)
images.sort()
segmentations  = os.listdir(dir_seg_test)
segmentations.sort()
    
X_test = []
Y_test = []
for im , seg in zip(images,segmentations) :
    X_test.append( getImageArr(dir_img_test + im , input_width , input_height )  )
    Y_test.append( getSegmentationArr( dir_seg_test + seg , n_classes , output_width , output_height )  )

X_test, Y_test = np.array(X_test) , np.array(Y_test)
#X_test = preprocess_input(X_test)
print(X_test.shape,Y_test.shape)

In [ ]:
np.min(X_test)

In [ ]:
y_pred = model.predict(X_test, batch_size=4, verbose=2)
y_predi = np.argmax(y_pred, axis=3)
y_testi = np.argmax(Y_test, axis=3)
print(y_testi.shape,y_predi.shape)

In [ ]:
fheight = output_height
fwidth = output_width
print (fwidth , fheight)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('FCN_last.avi',fourcc, 20.0, (fwidth,fheight))

for k in  range(349):
    img = (X_test[k] + 1)*(255.0/2)
    img = cv2.resize(img, ( output_width , output_height))

    mask_img = y_predi[k]*255
       
    for i in range(mask_img.shape[0]-2):
        for j in range (mask_img.shape[1]-2):
            if mask_img[i,j]> img[i,j,1]:
                img[i,j,1] = mask_img[i,j]
                if  img[i,j,0]> 50:
                    img[i,j,0] = 50
                #elif boxes_img[i,j,2]> 100:
                #     boxes_img[i,j,2] = 100   
    #img = np.uint8(255 * img) 
    img = np.array(img, dtype='uint8')
    #img = cv2.resize(img , (fwidth,fheight))   
    out.write(img)
    cv2.imshow('IMG ', img)
    k = cv2.waitKey(20)
    if k == 27: 
        break
        
        
cv2.destroyAllWindows()
print ('out')

In [ ]:
mask_img.shape 

In [ ]:
def IoU(Yi,y_predi):
    ## mean Intersection over Union
    ## Mean IoU = TP/(FN + TP + FP)

    IoUs = []
    Precision = []
    Recall = []
    F_Score = []
    
    Nclass = int(np.max(Yi)) + 1
    for c in range(Nclass):
        TP = np.sum( (Yi == c)&(y_predi==c) )
        FP = np.sum( (Yi != c)&(y_predi==c) )
        FN = np.sum( (Yi == c)&(y_predi != c)) 
        TN = np.sum( (Yi != c)&(y_predi != c))
        IoU = TP/float(TP + FP + FN)
        precision = TP/float(TP + FP)
        recall = TP/float(TP + FN)
        f_score =  2*((precision * recall)/(precision + recall))
        print("class {:02.0f}: #TP={:6.0f}, #FP={:6.0f}, #FN={:5.0f}, IoU={:4.3f}\Precision={:4.3f}  ,Recall={:4.3f}  ,F_Score={:4.3f}".format(c,TP,FP,FN,IoU,precision,recall,f_score))
        IoUs.append(IoU)
        Precision.append(precision)
        Recall.append(recall)
        F_Score.append(f_score)
    mIoU = np.mean(IoUs)
    mPrecision = np.mean(Precision)
    mRecall = np.mean(Recall)
    mF_Score = np.mean(F_Score)
    print("_________________")
    print("Mean IoU: {:4.3f}".format(mIoU)) 
    print("_________________")
    print("Mean Precision: {:4.3f}".format(mPrecision))
    print("_________________")
    print("Mean Recall: {:4.3f}".format(mRecall))
    print("_________________")
    print("Mean F_Score: {:4.3f}".format(mF_Score))


IoU(y_testi,y_predi)

In [ ]:
for i in range(30):
    img_is  = (X_test[i] + 1)*(255.0/2)
    seg = y_predi[i]
    segtest = y_testi[i]

    fig = plt.figure(figsize=(30,90))    
    ax = fig.add_subplot(1,3,1)
    ax.imshow(img_is/255.0)
    ax.set_title("original")
    
    ax = fig.add_subplot(1,3,2)
    ax.imshow(give_color_to_seg_img(seg,n_classes))
    ax.set_title("predicted class")
    
    ax = fig.add_subplot(1,3,3)
    ax.imshow(give_color_to_seg_img(segtest,n_classes))
    ax.set_title("true class")
    plt.show()